## SQL Server

In [1]:
import sqlite3
import pandas as pd
import re #Para quitar y solo mantener caracteres letras
conn = sqlite3.connect("Proyecto Empresa Aliada.db")
cur = conn.cursor()

## Importación de CSV y EXCEL

In [2]:
#Excels importaciones
df_Calendar = pd.read_excel("DIM_CALENDAR.xlsx") # Importacion de la base de datos
df_Product = pd.read_excel("DIM_PRODUCT.xlsx")
df_Segment = pd.read_excel("DIM_SEGMENT.xlsx")

#Csv importaciones
df_Category = pd.read_csv("DIM_CATEGORY.csv")
df_Sales = pd.read_csv("FACT_SALES.csv")

## Tabla date que contiene las semanas, año, mes, qué número de la semana seria del año y la fecha entera

In [3]:
df_Calendar["DATE"] = df_Calendar["DATE"].astype(str)
df_Calendar_Values = df_Calendar.values.tolist()

cur.execute("""CREATE TABLE IF NOT EXISTS Calendar (
            Week TEXT,
            Year INTEGER,
            Month INTEGER,
            Week_Number INTEGER,
            DateE DATE 
            );
""")

for renglon in range(len(df_Calendar_Values)):
    cur.execute(""" INSERT INTO Calendar(Week, Year, Month, Week_Number, DateE)
                VALUES(?,?,?,?,?)""", df_Calendar_Values[renglon])
conn.commit()

## Tabla Producto con el manufacturador, marca, nombre del objeto, descripción del objeto, categoria, formato y 3 distintos atributos

In [4]:
#Remover cualquier codigo de producto invalido en PRODUCTOS
ValoresNAN_Products = df_Product["ITEM"].isna() # Array booleano para eliminar los NaN
df_Product_N = df_Product[~ValoresNAN_Products] # Crear un array sin los Nan
df_Product_N=df_Product_N.reset_index(drop=True) #Borrar index previo y eliminar la columna del index


df_Product_Values = df_Product_N.values.tolist()
cur.execute("""CREATE TABLE IF NOT EXISTS Product (
            Manufacturer TEXT,
            Brand TEXT,
            Item TEXT,
            Item_Description TEXT,
            Category INTEGER,
            Format_t TEXT,
            ATTR1 TEXT,
            ATTR2 TEXT,
            ATTR3 TEXT
            );
""")

for renglon in range(len(df_Product_Values)):
    cur.execute(""" INSERT INTO Product(Manufacturer, Brand, ITEM,
                                         Item_Description, Category, Format_t,
                                         ATTR1, ATTR2, ATTR3)
                VALUES(?,?,? ,?,?,? ,?,?,?)""", df_Product_Values[renglon])
conn.commit()

## Tabla Segmentos con Categoría, 3 distintos atributos, el formato y segmento

In [5]:
df_Segment_Values = df_Segment.values.tolist()
cur.execute("""CREATE TABLE IF NOT EXISTS Segment (
            Category TEXT,
            ATTR1 TEXT,
            ATTR2 TEXT,
            ATTR3 TEXT,
            Format_t TEXT,
            Segment TEXT
            );
""")

for renglon in range(len(df_Segment_Values)):
    cur.execute(""" INSERT INTO Segment(Category, ATTR1, ATTR2,
                                         Format_t, ATTR3, Segment)
                VALUES(?,?,? ,?,?,?)""", df_Segment_Values[renglon])
conn.commit()

## Tabla con Categoria (ID) y el titulo de la categoría

In [6]:
#Tiene Caracteres CATEGORY como /r y /n, asi que se ocupan remover
for posicionCateg in range(len(df_Category["CATEGORY"])): # For para revisar cada posicion
    df_Category.at[posicionCateg,"CATEGORY"]= re.split('[^a-zA-Z ]',df_Category["CATEGORY"][posicionCateg])[0] #Reescribe la posicion con el contenido
             #del split (division de texto) antes de cualquier caracter que no sea letra (Mayuscula o minuscula) o especio
    df_Category.at[posicionCateg,"CATEGORY"]= df_Category["CATEGORY"][posicionCateg].upper() #Hace que todo sea mayusculas (no se hizo antes para NO
             # confundir que hace cada linea)

df_Category_Values = df_Category.values.tolist()
cur.execute("""CREATE TABLE IF NOT EXISTS Category (
            ID_Category INTEGER PRIMARY KEY,
            Category TEXT
            );
""")

for renglon in range(len(df_Category_Values)):
    cur.execute(""" INSERT INTO Category(ID_Category, Category)
                VALUES(?,?)""", df_Category_Values[renglon])
conn.commit()

## Tabla de ventas en los que indica la semana de venta, código del producto, unidades vendidas, valor vendido, promedio de unidades vendidas en la semana y región en la que se vendio

In [7]:
#Remover cualquier codigo de producto invalido en SALES
ValoresNAN_Sales = df_Sales["ITEM_CODE"].isna() # Array booleano para eliminar los NaN
df_Sales_N = df_Sales[~ValoresNAN_Sales] # Crear un array sin los Nan
df_Sales_N=df_Sales_N.reset_index(drop=True) #Borrar index previo y eliminar la columna del index

df_Sales_Values = df_Sales_N.values.tolist()
cur.execute("""CREATE TABLE IF NOT EXISTS Sales (
            Week TEXT,
            Item_code TEXT,
            Total_Unit_Sales FLOAT,
            Total_Value_Sales FLOAT,
            Total_Unit_AVG_Weekly_Sales FLOAT,
            Region TEXT
            );
""")

for renglon in range(len(df_Sales_Values)):
    cur.execute(""" INSERT INTO Sales(Week, Item_code, Total_Unit_Sales,
                                      Total_Value_Sales, Total_Unit_AVG_Weekly_Sales, Region)
                VALUES(?,?,?, ?,?,?)""", df_Sales_Values[renglon])
conn.commit()

## Volcado de base de datos

In [8]:
_file = open("CreateTablesPF.sql","w", encoding='utf-8')
for line in conn.iterdump():
    #print(line)
    _file.write(f"{line}\n")
_file.close()

In [9]:
conn.close()